In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
(I-sqrt(3)).minpoly()

x^4 - 4*x^2 + 16

In [3]:
var('nv')
K1.<i_minus_rad3> = NumberField(nv^4-4*nv^2+16)

In [4]:
i = i_minus_rad3^3/8
rad3 = (i_minus_rad3^3 - 8*i_minus_rad3)/8

In [5]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(K1)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
# Define cubic SE9, find and classify the lines and compute the 45 tritangent planes

In [7]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : i*rad3*c})

In [8]:
# Find simmetries and study possible subfamilies

In [9]:
%time adm_SE9 = SE9.find_admissible_projectivities()

CPU times: user 4.89 s, sys: 173 ms, total: 5.06 s
Wall time: 48.6 s


In [10]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)

CPU times: user 5.71 s, sys: 112 ms, total: 5.82 s
Wall time: 7.96 s


In [11]:
%time conds = SE9.find_conditions_for_subfamilies(adm_SE9, simm_SE9)

CPU times: user 2min 14s, sys: 708 ms, total: 2min 14s
Wall time: 2min 16s


In [12]:
sols = [solve_linear_system(q, [f], [c]) for q in conds] 
for sol in sols:
    print(sol[0]//sol[1]*c)

(-i_minus_rad3 + 1)*c
(-1/4*i_minus_rad3^3 - 1/2*i_minus_rad3^2 + 1)*c
(i_minus_rad3 + 1)*c
(1/4*i_minus_rad3^3 - i_minus_rad3 - 1)*c
(-1/4*i_minus_rad3^3 + i_minus_rad3 - 1)*c
(1/4*i_minus_rad3^3 - 1/2*i_minus_rad3^2 + 1)*c


In [22]:
[(1/4*(I-sqrt(3))^3 -(I-sqrt(3)) -1).expand(), (-1/4*(I-sqrt(3))^3 - 1/2 * (I-sqrt(3))^2 +1).expand(),
      (-1/4*(I-sqrt(3))^3 +(I-sqrt(3)) -1).expand(), (1/4*(I-sqrt(3))^3 - 1/2 * (I-sqrt(3))^2 +1).expand(),
      (-(I-sqrt(3)) +1).expand(), ((I-sqrt(3)) +1).expand()]

[sqrt(3) + I - 1,
 I*sqrt(3) - 2*I,
 -sqrt(3) - I - 1,
 I*sqrt(3) + 2*I,
 sqrt(3) - I + 1,
 -sqrt(3) + I + 1]

In [17]:
# Choose one subfamily (say SE9') and study it

In [13]:
SE9_1 = SE9.subs({f:(1/4*i_minus_rad3^3 - i_minus_rad3 - 1)*c})

In [14]:
%time adm_SE9_1 = SE9_1.find_admissible_projectivities()
len(adm_SE9_1)

CPU times: user 2.7 s, sys: 200 ms, total: 2.9 s
Wall time: 27.2 s


1296

In [15]:
simm_SE9_1 = SE9_1.find_simmetries(adm_SE9_1)
len(simm_SE9_1)

108

In [16]:
# Study how the simmetries permute the Eckardt points

In [17]:
Eck_points_perms = []
for proj in simm_SE9_1:
    Eck_points_perms.append(Permutation(SE9_1.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(36, '(C3 x C3) : C4')

In [18]:
# Study how the simmetries permute the 27 lines

In [19]:
lines_perms = []
for simm in simm_SE9_1:
    lines_perms.append(Permutation(SE9_1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(108, '((C3 x C3) : C3) : C4')